In [1]:
import os
import sys
import json
import re

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import custom_cde as cde
from custom_cde import Document
from custom_cde.doc import Paragraph, Heading
from custom_cde.reader import AcsHtmlReader
# from custom_cde.model import Compound
# from custom_cde.model import BaseModel, StringType, ListType, ModelType
# from custom_cde.parse import R, I, W, Optional, merge
# from custom_cde.parse.base import BaseParser
# from custom_cde.utils import first

%load_ext autoreload

In [17]:
d = Document(
#     Heading('Molecular energy levels of poly(3-hexylthiophene) (P3HT)'),
#     Paragraph('P3HT has a HOMO level of -4.9 eV, while the LUMO level is -3.0 eV. This corresponds to a bandgap of 1.9 eV.'),
#     Paragraph('For P3HT, EHOMO is -4.9 to -5.2 eV'),
#     Paragraph('For P3HT, ELUMO is -3.0 to -2.9 eV'),
#     Paragraph('for P3ht, the EBandGap is 1.9 to 2.3 eV'),
    
#     #what about the following sentence doesn't work, compared to the next one?
#     Paragraph(u'The gold electrode has a Fermi energy level of 5.1 eV'),
#     Paragraph(u'For gold, the Fermi energy level = 5.1 eV'),
    
#     Heading('Synthesis of 2,4,6-trinitrotoluene (3a)'),
#     Paragraph('The procedure was followed to yield a pale yellow solid (b.p. 240 °C)'),
    
    Heading('New property phrases'),
#     Paragraph('the crystallization temperature of gold was found to be ~1600 °C.'),
    Paragraph('for gold, the Tm is ~1600 °C.'),
#     Paragraph('The melting point of P3HT is 275 ºC.'),
#     Paragraph('The Tm of P3HT is 275 ºC.'),
#     Paragraph('the boiling point of niacin is 100 ºC.'),
#     Paragraph('For niacinamide, the corrosion inhibition efficiency = 90%.'),
#     Paragraph('for P3HT, the enthalpy of sublimation is 100 kJ/g'),
#     Paragraph('for P3HT, the enthalpy of vaporization is 73 J/g'),
    Paragraph('the enthalpy of crystallization of P3HT is 27 J/g'),
    Paragraph('for P3HT, the M_n is 30 kDa'),
#     Paragraph('for P3HT, the Đ is 1.2.'),
#     Paragraph('For P3HT, the modulus is between 1 and 5 GPa'),
    Paragraph('for RR-P3HT, the degree of crystallinity is 45%'),
    Paragraph('for RR-P3HT, the photoluminescence quantum yield is 45'),
)
d

<Document: 6 elements>

In [18]:
%autoreload

d.records.serialize()

[{'names': ['gold']},
 {'melting_points': [{'value': '~1600', 'units': '°C'}]},
 {'names': ['P3HT'], 'M_n': [{'value': '30', 'units': 'kDa'}]},
 {'names': ['RR-P3HT'], 'crystallinity': [{'value': '45', 'units': '%'}]}]

In [2]:
with open('/Users/wesleytatum/Desktop/10-1021-ma902164q.htm', 'rb') as f:
    acs_doc = Document.from_file(f, readers = [AcsHtmlReader()])
    f.close()

In [21]:
acs_doc

<Document: 350 elements>

In [11]:
acs_doc.serialize()

{'type': 'document',
 'elements': [{'type': 'Paragraph',
   'content': 'Download Hi-Res ImageDownload to MS-PowerPointCite This:Macromolecules 2010, 43, 2, 797-804'},
  {'type': 'Paragraph', 'content': '\nADVERTISEMENT\n'},
  {'type': 'Paragraph', 'content': 'RETURN TO ISSUEPREVArticleNEXT'},
  {'type': 'Paragraph',
   'content': 'Polycyclic Aromatics with Flanking Thiophenes: Tuning Energy Level and Band Gap of Conjugated Polymers for Bulk Heterojunction Photovoltaics'},
  {'type': 'Paragraph', 'content': 'Samuel C. Price'},
  {'type': 'Paragraph', 'content': '\n,\xa0\n'},
  {'type': 'Paragraph', 'content': 'Andrew C. Stuart'},
  {'type': 'Paragraph', 'content': '\n,\xa0and\xa0\n'},
  {'type': 'Paragraph', 'content': 'Wei You*'},
  {'type': 'Paragraph', 'content': 'View Author Information'},
  {'type': 'Paragraph',
   'content': 'Department of Chemistry, University of North Carolina at Chapel Hill, Chapel Hill, North Carolina 27599-3290'},
  {'type': 'Paragraph',
   'content': '*To wh

In [3]:
detected = acs_doc.records.serialize()
detected

[{'names': ['Wei You*']},
 {'names': ['Linked In']},
 {'names': ['fullerene']},
 {'names': ['benzo[1,2-b:4,5-b′]dithiophene (DTBn),(20)']},
 {'names': ['dithienopyridines']},
 {'names': ['n-dodecyl']},
 {'names': ['monostannyl']},
 {'names': ['deuterated methanol']},
 {'names': ['8-alkyldithienopyridines']},
 {'names': ['thiophene-3-carboxylic acid']},
 {'names': ['amine']},
 {'names': ['TFA']},
 {'names': ['2,5-bis(trimethyltin)thiophene']},
 {'names': ['2,1,3-benzothiadiazole-4,7-bis(boronic acid pinacol ester )']},
 {'names': ['PDTBn']},
 {'names': ['PDTBn-T,(25-27)']},
 {'names': ['carbon']},
 {'names': ['P3HT']},
 {'names': ['DTPn-BT']},
 {'names': ['DTPr-T']},
 {'names': ['tetramethylsilane']},
 {'names': ['o-dichlorobenzene']},
 {'names': ['BHT']},
 {'names': ['Pt']},
 {'names': ['tetrabutylammonium hexafluorophosphate']},
 {'names': ['ferrocene']},
 {'names': ['ferrocenium']},
 {'names': ['indium']},
 {'names': ['tin oxide', 'ITO']},
 {'names': ['acetone']},
 {'names': ['2-prop

In [8]:
recs_with_props = []
mps = []

for rec in detected:
    if len(rec.values()) > 1:
        print(rec)
        print('\n')
        recs_with_props.append(rec)
        compound = ''
        for name in rec['names']:
            compound += name + ', '
        for prop in rec.items():
        #props are tuples of the name and a list or dict of values
            if prop[0] == 'bandgaps':
                prop_mp = {compound : prop[1]}
                mps.append(prop_mp)
        
# recs_with_props
mps

{'names': ['PDTPr-T'], 'uvvis_spectra': [{'solvent': 'CHCl3', 'peaks': [{'value': '575', 'units': 'nm'}]}, {'solvent': 'CHCl3', 'peaks': [{'value': '659'}]}, {'peaks': [{'value': '609', 'units': 'nm'}]}, {'peaks': [{'value': '663'}]}]}


{'names': ['PDTBn-T'], 'uvvis_spectra': [{'solvent': 'CHCl3', 'peaks': [{'value': '537', 'units': 'nm'}]}, {'solvent': 'CHCl3', 'peaks': [{'value': '566'}]}, {'peaks': [{'value': '543', 'units': 'nm'}]}, {'peaks': [{'value': '584'}]}]}


{'names': ['dithiophene', 'Dithiophene'], 'roles': ['product']}


{'names': ['PDTPn-T'], 'uvvis_spectra': [{'solvent': 'CHCl3', 'peaks': [{'value': '465', 'units': 'nm'}]}, {'solvent': 'CHCl3', 'peaks': [{'value': '535'}]}, {'peaks': [{'value': '465', 'units': 'nm'}]}, {'peaks': [{'value': '534'}]}]}


{'names': ['PDTPn-BT'], 'uvvis_spectra': [{'solvent': 'CHCl3', 'peaks': [{'value': '508', 'units': 'nm'}]}, {'solvent': 'CHCl3', 'peaks': [{'value': '580'}]}, {'peaks': [{'value': '540', 'units': 'nm'}]}, {'peaks': [{'va

[]